In [1]:
import cPickle as pkl
import pandas as pd
import numpy as np
import os.path

In [2]:
dirname = '../../results_20180425_23/'

In [3]:
files = !ls {dirname}
files = [os.path.join(dirname, filename) for filename in files]
files

['../../results_20180425_23/test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 100 ribosomes, deltat=0.1s_20180425_0943_100_ribosomes_0200s.p',
 '../../results_20180425_23/test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 10 ribosomes, deltat=0.1s_20180425_0943_10_ribosomes_0200s.p',
 '../../results_20180425_23/test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 1 ribosomes, deltat=0.1s_20180425_0943_1_ribosomes_0200s.p',
 '../../results_20180425_23/test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 200 ribosomes, deltat=0.1s_20180425_0943_200_ribosomes_0200s.p',
 '../../results_20180425_23/test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 20 ribosomes, deltat=0.1s_20180425_0943_20_ribosomes_0200s.p',
 '../../results_20180425_23/test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 2 ribosomes, deltat=0.1s_20180425_0943_2_ribosomes_0200s.p',
 '../../re

In [4]:
descriptor = files[0].split('/')[-1][:-2]
descriptor

'test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 100 ribosomes, deltat=0.1s_20180425_0943_100_ribosomes_0200s'

Blacklisted species:

In [5]:
blacklist = ['ribos._bound', 'ribos._free', 'tRNA_free', 'tRNA_bound', 'ATP', 'GTP', 'AMP', 'GDP', 'protein', 
             'proteins', 'peptide_bonds']

Parameters for efficiency:

In [6]:
sim_time = 200.
timepoints = sim_time / 0.1
print timepoints

2000.0


In [7]:
post_burn_in = 100.  # seconds
share_of_total = post_burn_in / sim_time
print share_of_total

0.5


In [8]:
points_for_efficiency = int(share_of_total * timepoints + 1)
print points_for_efficiency

1001


Data frame at cell level:

In [9]:
dfc = pd.DataFrame(columns=['ribosomes', 'transcripts', 'collisions', 'nocollisions', 'peptide_bonds'])

In [10]:
rows = []
dft_columns = ['length', 'init_rate', 'ribosomes', 'tic_toc', 'geneID', 'transcripts', 'index']

for i, filename in enumerate(files):
    print filename
    results = pkl.load(open(filename))
    
    # append to cell level data frame
    print 'cell level...'
    rowvals = [results['n_ribosomes'], len(results['transcriptome']), results['collisions'], 
               results['nocollisions'], results['peptide_bonds']]
    row = dict(zip(list(dfc.columns), rowvals))
    #print row
    dfc = dfc.append(row, ignore_index=True)
    #print dfc.head()
    
    # append to transcript level data frame
    print 'transcript level...'
    
    for transcript in results['transcriptome']:
        av_time = np.mean([tictoc[1] - tictoc[0] for tictoc in transcript.tic_toc])
        rowvals = [transcript.geneID, transcript.index, transcript.init_rate, transcript.length, 
                   results['n_ribosomes'], av_time, len(results['transcriptome'])]
        row = dict(zip(list(dft_columns), rowvals))
        #print row
        rows.append(row)
            
    # append to gene level data frame
    print 'gene level...'
    header = str(results['n_ribosomes']) + '_' + str(len(results['transcriptome']))
    tc = results['timecourses']
    efficiency_items = [(gene, (tc[gene][-1] - tc[gene][-points_for_efficiency]) / post_burn_in) for gene in tc 
                        if gene not in blacklist]
    tmp = pd.DataFrame(efficiency_items, columns = ['gene', header]).set_index('gene')
    
    if i == 0:  # first time
        dfg = tmp
    else:
        dfg = pd.merge(dfg, tmp, left_index=True, right_index=True, how='outer')

dfc.sort_values('ribosomes', inplace=True)

dft = pd.DataFrame(rows)
dft.columns = ['ribosomes', 'transcripts', 'index', 'geneID', 'init_rate', 'length', 'tic_toc']

../../results_20180425_23/test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 100 ribosomes, deltat=0.1s_20180425_0943_100_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180425_23/test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 10 ribosomes, deltat=0.1s_20180425_0943_10_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180425_23/test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 1 ribosomes, deltat=0.1s_20180425_0943_1_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180425_23/test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 200 ribosomes, deltat=0.1s_20180425_0943_200_ribosomes_0200s.p


/home/martin/.local/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/martin/.local/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


cell level...
transcript level...
gene level...
../../results_20180425_23/test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 20 ribosomes, deltat=0.1s_20180425_0943_20_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180425_23/test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 2 ribosomes, deltat=0.1s_20180425_0943_2_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180425_23/test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 30 ribosomes, deltat=0.1s_20180425_0943_30_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180425_23/test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 3 ribosomes, deltat=0.1s_20180425_0943_3_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180425_23/test configuration with 2 genes, transcripts 4, 2, init rates 0.001, 0.1, 500 ribosomes, delta

In [11]:
dfc

,ribosomes,transcripts,collisions,nocollisions,peptide_bonds
2,1.0,6.0,0.0,16.0,496.0
5,2.0,6.0,0.0,32.0,948.0
7,3.0,6.0,10.0,46.0,1378.0
10,5.0,6.0,35.0,68.0,2034.0
12,8.0,6.0,119.0,95.0,2806.0
1,10.0,6.0,155.0,115.0,3474.0
4,20.0,6.0,495.0,138.0,3989.0
6,30.0,6.0,920.0,152.0,4209.0
9,50.0,6.0,1642.0,166.0,4412.0
11,80.0,6.0,2821.0,183.0,4758.0


In [12]:
dft = dft.dropna(subset=['tic_toc'])
dft

,ribosomes,transcripts,index,geneID,init_rate,length,tic_toc
0,100,6,0,0,0.001,36,1.863636
1,100,6,1,0,0.001,36,2.458333
2,100,6,2,0,0.001,36,1.800000
3,100,6,3,0,0.001,36,2.023529
4,100,6,4,1,0.100,96,7.454545
5,100,6,5,1,0.100,96,6.848000
9,10,6,3,0,0.001,36,2.600000
10,10,6,4,1,0.100,96,6.400000
11,10,6,5,1,0.100,96,6.854545
16,1,6,4,1,0.100,96,7.240000


In [13]:
dfg

,100_6,10_6,1_6,200_6,20_6,2_6,30_6,3_6,500_6,50_6,5_6,80_6,8_6
gene,,,,,,,,,,,,,
0,0.27,0.01,NaN,0.59,0.04,0.01,0.11,0.00,0.93,0.20,0.01,0.21,0.01
1,0.71,0.54,0.08,0.70,0.65,0.14,0.66,0.23,0.74,0.64,0.37,0.69,0.44
tRNA_free_01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_07,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [14]:
dfc.to_pickle(dirname + '/dfc' + descriptor + '.p')

In [15]:
dft.to_pickle(dirname + '/dft' + descriptor + '.p')

In [16]:
dfg.to_pickle(dirname + '/dfg' + descriptor + '.p')

Here we start the analysis (old notebook 17):

In [17]:
dfc['collisions_percentage'] = dfc['collisions'] / (dfc['collisions'] + dfc['nocollisions'])

In [18]:
dfc

,ribosomes,transcripts,collisions,nocollisions,peptide_bonds,collisions_percentage
2,1.0,6.0,0.0,16.0,496.0,0.000000
5,2.0,6.0,0.0,32.0,948.0,0.000000
7,3.0,6.0,10.0,46.0,1378.0,0.178571
10,5.0,6.0,35.0,68.0,2034.0,0.339806
12,8.0,6.0,119.0,95.0,2806.0,0.556075
1,10.0,6.0,155.0,115.0,3474.0,0.574074
4,20.0,6.0,495.0,138.0,3989.0,0.781991
6,30.0,6.0,920.0,152.0,4209.0,0.858209
9,50.0,6.0,1642.0,166.0,4412.0,0.908186
11,80.0,6.0,2821.0,183.0,4758.0,0.939081


In [19]:
ax = dfc.plot(x='ribosomes', y='collisions_percentage', figsize=(10, 7), fontsize=16)
ax.legend(["5 transcripts"], prop={'size': 16})
#plt.show()

In [20]:
dft_grouped = dft.groupby(['geneID', 'init_rate', 'length', 'ribosomes', 'transcripts'], as_index=False).mean()
dft_grouped

,geneID,init_rate,length,ribosomes,transcripts,index,tic_toc
0,0,0.001,36,2,6,3.0,3.300000
1,0,0.001,36,3,6,2.0,2.200000
2,0,0.001,36,5,6,2.0,1.600000
3,0,0.001,36,8,6,1.5,2.400000
4,0,0.001,36,10,6,3.0,2.600000
5,0,0.001,36,20,6,1.5,2.000000
6,0,0.001,36,30,6,1.5,2.244643
7,0,0.001,36,50,6,1.5,2.107857
8,0,0.001,36,80,6,1.5,2.162917
9,0,0.001,36,100,6,1.5,2.036375
